## crawling한 korean books에서 중복되는 책들을 삭제, 에디션만 다른 동일한 내용의 책 삭제

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/14 14:46:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/14 14:46:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT title, author FROM Korean_book")
data = cur.fetchall()
print(len(data))
conn.commit()
conn.close()


44945


In [3]:
## 중복 도서 삭제 확인
rdd = sc.parallelize(data)
S = set({})
D = set({})
for d in rdd.collect():
    if d in S:
        D.add(d[0])
    else:
        S.add(d[0])

print(len(S))
print(len(D)) # <--- 실제 중복 데이터(책 제목과 글쓴이도 다른)
real_dup_data = list(D)

44616
0


In [ ]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()

# 하나 빼고 중복되는 도서들 모두 삭제
for i in range(len(real_dup_data)):
    cur.execute(f"SELECT rowid FROM Korean_book WHERE title=\"{real_dup_data[i]}\"")
    same = cur.fetchall()
    del_id = []
    for num, (item) in enumerate(same):
        if num==0:
            pass
        else:
            del_id.append(item[0])

    for id in del_id:
        cur.execute(f"DELETE FROM Korean_book WHERE rowid={id}")
    conn.commit()
    cur.execute(f"SELECT rowid FROM Korean_book WHERE title=\"{real_dup_data[i]}\"")
    same = cur.fetchall()

conn.close()

In [108]:
## 이름은 같지만 에디션, 큰글씨, 세트 등 부가사항들일 붙어있는 도서 삭제
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT rowid, title, author FROM Korean_book")
data = cur.fetchall()
# print(data[0:3])

dup = {}
count = 0
for id, title, author in data:
    if "(큰글" in title or "(리커버)" in title or "(초판본)" in title\
        or "에디션)" in title or "커버)" in title:
        # cur.execute(f"DELETE FROM Korean_book WHERE rowid=={id}")
        print(id, title, author)
        count +=1
print(count)
conn.commit()
cur.close()
conn.close()

1 하얼빈 김훈
2 아버지의 해방일지 정지아
3 불편한 편의점 2 김호연
4 불편한 편의점(40만부 기념 벚꽃 에디션) 김호연
5 작별인사(밤하늘 에디션) 김영하
6 어서 오세요, 휴남동 서점입니다 (여름 숲 에디션) 황보름
7 모순 양귀자
8 이토록 평범한 미래 김연수
9 달러구트 꿈 백화점(레인보우 에디션) 이미예
11 데뷔 못 하면 죽는 병 걸림 1부 초판 한정 굿즈박스 세트 백덕수
12 달러구트 꿈 백화점 2(레인보우 에디션) 이미예
13 재수사 1 장강명
14 보테로 가족의 사랑 약국 이선영
15 김승옥문학상 수상작품집(2022) 편혜영 외
16 밝은 밤 최은영
17 가녀장의 시대 이슬아
18 지구 끝의 온실(여름 에디션) 김초엽
19 재수사 2 장강명
20 튜브 손원평
21 세계를 건너 너에게 갈게 이꽃님
22 데뷔 못 하면 죽는 병 걸림 1부 초판 세트 백덕수
24 대도시의 사랑법 박상영
25 소설 보다: 가을 2022 김기태 외
26 트로피컬 나이트 조예은
29 체리새우: 비밀글입니다 황영미
30 아몬드 손원평
31 구의 증명 최진영
32 우리가 빛의 속도로 갈 수 없다면 김초엽
33 순례 주택 유은실
34 클로버 나혜림
35 아라의 소설 정세랑
36 오백 년째 열다섯 김혜정
37 시선으로부터, 정세랑
38 완전한 행복 정유정
39 파과 구병모
40 나는 소망한다 내게 금지된 것을 양귀자
41 천 개의 파랑 천선란
42 죽이고 싶은 아이 이꽃님
43 칼의 노래 김훈
44 사서함 110호의 우편물 이도우
45 시간이 없다 정찬주
47 채식주의자 한강
48 친밀한 이방인 정한아
49 가짜 모범생 손현주
50 망원동 브라더스 김호연
51 믿음에 대하여 박상영
52 작별하지 않는다 한강
53 소년이 온다 한강
54 눈감지 마라 이기호
55 종의 기원 정유정
56 2022 제13회 젊은작가상 수상작품집 임솔아 외
57 저만치 혼자서 김훈
58 시간을 파는 상점 김선영
59 저주토끼 정보라
60 쇼코의 미소 최은영
61 하쿠다 사진관 허태연
62 책들의 부

KeyboardInterrupt: 

In [96]:
## 이름은 같지만 에디션, 큰글씨, 세트 등 부가사항들일 붙어있는 도서 삭제
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("SELECT rowid, title, author FROM Korean_book")
data = cur.fetchall()
# print(data[0:3])

dup = {}
for id, title, author in data:
    for i, t, a in data:
        if id!=i and author==a:
            if (title in t) or (t in title):
                dup[author] = ((id,title),(i,t))
            else:
                pass

In [97]:
print(len(dup))
for author, line in zip(dup.keys(), dup.values()):
    print(author,":",line)

491
손원평 : ((30, '아몬드'), (10, '아몬드(100만 부 기념 특별판)'))
김선영 : ((44512, '시간을 파는 상점 2: 너를 위한 시간'), (58, '시간을 파는 상점'))
박현숙 : ((319, '구미호 식당 2: 저세상 오디션'), (104, '구미호 식당'))
김시습 : ((45159, '고전소설'), (45086, '우리고전소설 1(중고생이 읽어야 할 필독)'))
박완서 : ((235, '나목'), (225, '나목 도둑맞은 가난'))
이금이 : ((44697, '거기, 내가 가면 안 돼요? 2'), (247, '거기, 내가 가면 안 돼요?'))
황성혁 : ((254, '축복 2'), (248, '축복'))
일연 : ((46074, '청소년 삼국유사'), (265, '삼국유사'))
올더스 헉슬리 : ((43599, '멋진 신세계 (다락원 클리프노트)'), (506, '멋진 신세계'))
조지 오웰 : ((50501, '1984 1'), (511, '1984'))
어니스트 헤밍웨이 : ((50418, 'The Old Man and the Sea (노인과 바다)'), (514, '노인과 바다'))
허먼 멜빌 : ((4326, '일러스트 모비 딕'), (526, '모비 딕'))
제인 오스틴 : ((715, '오만과 편견(초판본)(스페셜 에디션)(1894년 오리지널 초판본 표지디자인)'), (528, '오만과 편견'))
헨리 데이비드 소로 : ((14972, '주석달린 월든'), (547, '월든'))
루시 모드 몽고메리 : ((995, '빨강 머리 앤(리커버 유화 에디션)'), (594, '빨강 머리 앤'))
루이자 메이 올컷 : ((920, '작은 아씨들 2'), (913, '작은 아씨들'))
리처드 바크 : ((987, '갈매기의 꿈'), (596, '갈매기의 꿈(완결판)'))
트리나 포올러스 : ((771, '꽃들에게 희망을'), (599, '꽃들에게 희망을(BESTSELLER WORLDBOOK 20)'))
윌리엄 셰익스피어 :

In [99]:
import sqlite3
conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
cur.execute("DELETE FROM Korean_book WHERE rowid==10")
cur.execute("SELECT rowid, author, title FROM Korean_book WHERE rowid==10")
data = cur.fetchall()
print(data)
conn.commit()
cur.close()
conn.close()

[]
